In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_squared_error

# Load the dataset
dataset = pd.read_csv('Coconut_Price_SriLanka_2005_2025.csv')

# Convert 'Month' column to datetime objects
dataset['Month'] = pd.to_datetime(dataset['Month'], format='%d/%m/%Y')

# Convert datetime objects to ordinal values (number of days since a reference date)
x = dataset['Month'].apply(lambda date: date.toordinal())
x = pd.DataFrame(x)

# Separate target variable
y = dataset["Coconut_Price"]

# Scale features and target
x_scaler = MinMaxScaler()
x_scaled = x_scaler.fit_transform(x)
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state = 42)

# Build the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, input_shape=(x_train.shape[1],), activation='sigmoid'))
model.add(tf.keras.layers.Dense(256, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(x_train, y_train, epochs=20, verbose=0)


# Evaluate the model
test_loss = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss (MSE): {test_loss:.4f}")

# Make predictions on the test set
y_pred_scaled = model.predict(x_test)

# Invert scaling to get predictions in the original scale
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_true = y_scaler.inverse_transform(y_test)

# Evaluate the predictions
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# If you want to make a new prediction:
new_date = pd.to_datetime('01/02/2028', format = '%d/%m/%Y')
new_date_ordinal = new_date.toordinal()

new_date_scaled = x_scaler.transform([[new_date_ordinal]])

# Predict the coconut price
predicted_price_scaled = model.predict(new_date_scaled)
predicted_price = y_scaler.inverse_transform(predicted_price_scaled)
print(f"Predicted Coconut Price for {new_date.strftime('%d/%m/%Y')}: {predicted_price[0][0]:.2f}")

model.save('Coconut_Price_model.h5')

from google.colab import files
files.download('Coconut_Price_model.h5')

Root Mean Squared Error (RMSE): 162.31
